<a href="https://colab.research.google.com/github/truongthuanr/transfer/blob/main/NLP_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CommonLit Readability problem

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

### Check for GPU

In [ ]:
!nvidia-smi -L

### Line wrap setting for notebook

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### Get helper function

In [ ]:
# Get the heper function 
!wget https://raw.githubusercontent.com/truongthuanr/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper function for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get text data set
The data set we going to using is from Kagle "CommonLit Readability Prize" Competetion
https://www.kaggle.com/c/commonlitreadabilityprize/data


In [ ]:
# Unzip data
# !unzip "/content/drive/MyDrive/01_Personal/01_Study/01_ML/02_CommonLit_Readability/commonlitreadabilityprize.zip" -d "/content/drive/MyDrive/01_Personal/01_Study/01_ML/02_CommonLit_Readability/"

## Visualizing a text dataset

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/01_Personal/01_Study/01_ML/02_CommonLit_Readability/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/01_Personal/01_Study/01_ML/02_CommonLit_Readability/test.csv")

In [ ]:
train_df.columns

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)

In [ ]:
# train_df_shuffled.head()

In [ ]:
# How the target look like
fig, axes = plt.subplots(1,2,figsize=(10, 5))
sns.kdeplot(train_df.target,color='navy',shade=True, ax=axes[0]) # Plot the target
sns.kdeplot(train_df.standard_error,color='lightseagreen',shade=True, ax=axes[1]) # Plot the sandard_error

# Set the title
axes[0].set_title('Target Distribution')
axes[1].set_title('Standard Error Distribution')

axes[0].axvline(train_df.target.mean(),color="navy",linestyle=':')
axes[1].axvline(train_df.standard_error.mean(),color="navy",linestyle=':')

axes[0].annotate('mean', xy=(-0.3* np.pi, 0.2), xytext=(1, 0.2),
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="angle3,angleA=0,angleB=-90"));
axes[1].annotate('mean', xy=(0.49, 6), xytext=(0.57, 6),
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="angle3,angleA=0,angleB=-90"));

In [ ]:
sns.jointplot(x=train_df.target, 
              y=train_df.standard_error,
              kind='hex',
              height=10,
              edgecolor='coral',
              color='mediumpurple')

In [ ]:
# How many total sample?
len(train_df), len(test_df)

In [ ]:
# visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-2)
for row in train_df_shuffled[["excerpt","target"]][random_index:random_index+2].itertuples():
  _, text, target = row
  print(f"Target: {target}")
  print(f"{text}\n")
  print("---")

In [ ]:
# Take a look on other column
plt.figure(figsize=(10,5))
# Use the count plot with horizontal bar due to long label of license
sns.countplot(data=train_df,y='license',palette='viridis')

plt.title('License Distribution')

## Split data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Use train_test_split to split trainind data into training and validation sets
train_excerpt, val_excerpt, train_target, val_target = train_test_split(train_df_shuffled["excerpt"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=41)

In [ ]:
len(train_excerpt), len(train_target), len(val_excerpt), len(val_target)

## Converting text in to number
When dealing with the text problem, one of the first things we have to do before you can build a model is to convert our text to number.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector canbe defined and hthis embedding can be learned)

### Text vectorization (Tokenization)

Info: https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization

In [ ]:
# train_excerpt[:4]

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many word in the vocabulary
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int", # how to map tokens to number
                                    output_sequence_length=None, # how long do we want our sequences to be
                                    pad_to_max_tokens=True)

In [ ]:
# Find the avarage number of tokens (words) in the training excerpt
round(sum([len(i.split()) for i in train_excerpt])/len(train_excerpt))

In [ ]:
# Setup the vectorization variables 
max_vocab_length = 10000 # max number of words to have in our vocabolary
max_length = 173 # max length our excerpt will be

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
# Fit the text Vectorizer to the text training data
text_vectorizer.adapt(train_excerpt)

In [ ]:
# Create a sample excerpt
sample_excerpt = '''\
The adjective "deep" in deep learning refers to the use of multiple layers in the network. \
Early work showed that a linear perceptron cannot be a universal classifier, but that a \
network with a nonpolynomial activation function with one hidden layer of unbounded width can. \
Deep learning is a modern variation which is concerned with an unbounded number of layers of \
bounded size, which permits practical application and optimized implementation, while retaining \
theoretical universality under mild conditions.
In deep learning, each level learns to transform its input data into a slightly more abstract and \
composite representation. In an image recognition application, the raw input may be a matrix of pixels; \
the first representational layer may abstract the pixels and encode edges; the second layer may \
compose and encode arrangements of edges; the third layer may encode a nose and eyes; \
and the fourth layer may recognize that the image contains a face. Importantly, a deep learning \
process can learn which features to optimally place in which level on its own.
'''

text_vectorizer([sample_excerpt])

In [ ]:
# Choose a random excerpt from the training dataset and tokenize it 
random_excerpt = random.choice(train_excerpt)
print(f"Origin excerpt:\n {random_excerpt}\
        \n\nVectorize version:")
text_vectorizer([random_excerpt])

In [ ]:
# Get the unique word in the vocabolary
words_in_vocab = text_vectorizer.get_vocabulary() # Get all of the unique words
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}" )
print(f"5 least common words: {bottom_5_words}")

top_common_word = words_in_vocab[:20]


### Creating an Embedding using an Embedding Layer

We've got a way to map our text to numbers. How about we go a step further and turn those numbers into an embedding?

The powerful thing about an embedding is it can be learned during training. This means rather than just being static (e.g. `1` = I, `2` = love, `3` = TensorFlow), a word's numeric representation can be improved as a model goes through data samples.

We can see what an embedding of a word looks like by using the [`tf.keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer. 

The main parameters we're concerned about here are:
* `input_dim` - The size of the vocabulary (e.g. `len(text_vectorizer.get_vocabulary()`).
* `output_dim` - The size of the output embedding vector, for example, a value of `100` outputs a  feature vector of size 100 for each word.
* `embeddings_initializer` - How to initialize the embeddings matrix, default is `"uniform"` which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.
* `input_length` - Length of sequences being passed to embedding layer.

Knowing these, let's make an embedding layer.

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length) # how long is each input

embedding

In [ ]:
# Get a random excerpt from the training set 
random_excerpt = random.choice(train_excerpt)
print(f"Origin excerpt:\n {random_excerpt}\
        \n\nEmbedded version:")

# Embed the random excerpt (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_excerpt]))
sample_embed

In [ ]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_excerpt

## Modeling a text dataset (running a series of experiments)

Now we've a got way to turn out text sequences into numbers, it's time to start building a series of modelling experiments.
We'll start with a baseline and move on from there. 

* **Model 0**: Naive Bayes (baseline)
* **Model 1**: Feed-forward neural network (dense model)
* **Model 2**: LSTM model
* **Model 3**: GRU model
* **Model 4**: Bidirectional-LSTM model
* **Model 5**: 1D Convolutional Neural Network
* **Model 6**: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* **Model 7**: Same as model 6 with 10% of training data

Use the standard step in modellign with tensorflow:
* Create a model
* Build a model
* Fit a model 
* Evaluate a model 


### Model 0: Geting a baseline
As with all machine learning modelling experiments, it's important to create a baselinemodel so you've got a benchmark for future experiments to build upon.

To create baseline, use sklearn's multinomial Naive Bayes using the TF-IDF formula to convert our words t onumbers. 

> **Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

# Create tokenization and modelling pipeline
model_0 = Pipeline([
            ("tfidf", TfidfVectorizer()), # Convert words to numbers using tfidf
            ("r", Ridge(alpha=0.5)) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_excerpt, train_target)

In [ ]:
predict = model_0.predict(val_excerpt)
from sklearn.metrics import mean_squared_error
import numpy as np
score = np.sqrt(mean_squared_error(val_target,predict))
score

### **Model 1:** A simple Dense Model

In [ ]:
# Create a tensorboard callback (need to craete a new one for each model)

from helper_functions import create_tensorboard_callback

# Create a dice